In [6]:
# import numpy as np
# import keras
# from keras import layers


# def readucr(filename):
#     data = np.loadtxt(filename, delimiter="\t")
#     y = data[:, 0]
#     x = data[:, 1:]
#     return x, y.astype(int)


# root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

# x_train, y_train = readucr(root_url + "FordA_TRAIN.tsv")
# x_test, y_test = readucr(root_url + "FordA_TEST.tsv")

# input_shape = x_train.shape[1:]
# print("Input_shape : ", input_shape)

# x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
# x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

# input_shape = x_train.shape[1:]
# print("Input_shape : ", input_shape)

# n_classes = len(np.unique(y_train))

# idx = np.random.permutation(len(x_train))
# x_train = x_train[idx]
# y_train = y_train[idx]

# input_shape = x_train.shape[1:]
# print("Input_shape : ", input_shape)

# y_train[y_train == -1] = 0
# y_test[y_test == -1] = 0

Input_shape :  (500,)
Input_shape :  (500, 1)
Input_shape :  (500, 1)


In [3]:
input_shape = x_train.shape[1:]
print("Input_shape : ", input_shape)

Input_shape :  (500, 1)


In [2]:
# from tensorflow import keras
# from tensorflow.keras import layers

# def tmr_transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
#     # Triple Encoders
#     encoders_outputs = []
#     for i in range(3):
#         # Attention and Normalization
#         encoder_input = inputs if i == 0 else encoders_outputs[i-1]
#         x = layers.MultiHeadAttention(
#             key_dim=head_size, num_heads=num_heads, dropout=dropout
#         )(encoder_input, encoder_input)
#         x = layers.Dropout(dropout)(x)
#         x = layers.LayerNormalization(epsilon=1e-6)(x)
#         res = x + encoder_input

#         # Feed Forward Part
#         x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
#         x = layers.Dropout(dropout)(x)
#         x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
#         x = layers.LayerNormalization(epsilon=1e-6)(x)
#         encoders_outputs.append(x + res)

#     # Majority Voter (Decoder)
#     voter_input = layers.Concatenate(axis=2)(encoders_outputs)
#     voter_input = layers.Dense(inputs.shape[-1], activation='relu')(voter_input)
#     voter_output = layers.Dense(inputs.shape[-1], activation='softmax')(voter_input)

#     # Updating Encoders
#     updated_encoders_outputs = []
#     for i in range(3):
#         x = layers.Add()([voter_output, encoders_outputs[i]])
#         updated_encoders_outputs.append(x)

#     return updated_encoders_outputs, voter_output

# def build_model(
#     input_shape,
#     head_size,
#     num_heads,
#     ff_dim,
#     num_transformer_blocks,
#     mlp_units,
#     n_classes,  # <- Here's where we add n_classes
#     dropout=0,
#     mlp_dropout=0,
# ):
#     inputs = keras.Input(shape=input_shape)
#     x = inputs
#     for _ in range(num_transformer_blocks):
#         encoders_outputs, _ = tmr_transformer_encoder(x, head_size, num_heads, ff_dim, dropout)
#         x = layers.Concatenate(axis=2)(encoders_outputs)

#     x = layers.GlobalAveragePooling1D(data_format="channels_last")(x)
#     for dim in mlp_units:
#         x = layers.Dense(dim, activation="relu")(x)
#         x = layers.Dropout(mlp_dropout)(x)
#     outputs = layers.Dense(n_classes, activation="softmax")(x)
#     return keras.Model(inputs, outputs)


In [3]:
# input_shape = x_train.shape[1:]
# num_classes = 2

# model = build_model(
#     input_shape=input_shape,  # Specify input_shape as a keyword argument
#     head_size=256,
#     num_heads=4,
#     ff_dim=4,
#     num_transformer_blocks=4,
#     mlp_units=[128],
#     mlp_dropout=0.4,
#     n_classes=num_classes,  # Assuming you have num_classes defined somewhere
#     dropout=0.25,
# )

# model.compile(
#     loss="sparse_categorical_crossentropy",
#     optimizer=keras.optimizers.Adam(learning_rate=1e-4),
#     metrics=["sparse_categorical_accuracy"],
# )
# model.summary()

# callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

# model.fit(
#     x_train,
#     y_train,
#     validation_split=0.2,
#     epochs=2,
#     batch_size=64,
#     callbacks=callbacks,
# )

# model.evaluate(x_test, y_test, verbose=1)


In [4]:
import numpy as np
import keras
from keras import layers
import pandas as pd
from sklearn.model_selection import train_test_split


# Load the CSV file
csv_file = "DeepIOD_Dataset.csv"
data = pd.read_csv(csv_file)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data.iloc[:, :-1], data.iloc[:, -1], test_size=0.2, random_state=42)

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))


def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res



def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_last")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)


input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=150,
    batch_size=64,
    callbacks=callbacks,
)

model.evaluate(x_test, y_test, verbose=1)

2024-03-27 02:28:39.430713: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-27 02:28:39.507690: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-27 02:28:39.507708: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-03-27 02:28:39.821472: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

IndexError: Exception encountered when calling layer 'softmax' (type Softmax).

tuple index out of range

Call arguments received by layer 'softmax' (type Softmax):
  • inputs=tf.Tensor(shape=(None, 4), dtype=float32)
  • mask=None

In [7]:
import numpy as np
import keras
from keras import layers
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the CSV file
csv_file = "DeepIOD_Dataset.csv"
data = pd.read_csv(csv_file)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data.iloc[:, :-1], data.iloc[:, -1], test_size=0.2, random_state=42)

n_classes = len(np.unique(y_train))  # Calculate the number of unique classes

# Define the input shape
input_shape = x_train.shape[1:]

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res

def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    n_classes,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_last")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)  # Use n_classes here
    return keras.Model(inputs, outputs)

# Define the number of classes
n_classes = len(np.unique(y_train))

# Define the input shape
input_shape = x_train.shape[1:]

# Create and compile the model
model = build_model(
    input_shape=input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    n_classes=n_classes,
    mlp_dropout=0.4,
    dropout=0.25,
)


model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)

model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=150,
    batch_size=64,
    callbacks=callbacks,
)

model.evaluate(x_test, y_test, verbose=1)


IndexError: Exception encountered when calling layer 'softmax' (type Softmax).

tuple index out of range

Call arguments received by layer 'softmax' (type Softmax):
  • inputs=tf.Tensor(shape=(None, 4), dtype=float32)
  • mask=None

In [9]:
import numpy as np
import keras
from keras import layers
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the CSV file
csv_file = "DeepIOD_Dataset.csv"
data = pd.read_csv(csv_file)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data.iloc[:, :-1], data.iloc[:, -1], test_size=0.2, random_state=42)

# Calculate the number of unique classes
n_classes = len(np.unique(y_train))

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res

def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    n_classes,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_last")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)

# Define the input shape
input_shape = x_train.shape[1:]
print("input_shape = ", input_shape)

# Create and compile the model
model = build_model(
    input_shape=input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    n_classes=n_classes,
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)

model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=150,
    batch_size=64,
    callbacks=callbacks,
)

model.evaluate(x_test, y_test, verbose=1)


input_shape =  (45,)


IndexError: Exception encountered when calling layer 'softmax' (type Softmax).

tuple index out of range

Call arguments received by layer 'softmax' (type Softmax):
  • inputs=tf.Tensor(shape=(None, 4), dtype=float32)
  • mask=None